In [40]:
#!pip install pyatmo

In [12]:
# Import des packages
import pyatmo
from datetime import datetime as dt
import datetime


In [13]:
# Import du package qui doit préalablement être installé


In [14]:
# Information sur l'application qui est à créer depuis https://dev.netatmo.com/
CLIENT_ID = str('606609b7b70a421ea72b545a')
CLIENT_SECRET = 'XPgT5THYut4ska9BaRtsf3MeXZ6jIUz1yOeoKZ'
# Mail et mot de passe renseignés lors de la connexion du capteur
USERNAME = 'marie-lise.pannier@univ-angers.fr'
PASSWORD = 'mdp-ProjetPEIP2'
# Adresse MAC du capteur disponible depuis les paramètres avancés de l'application Smartphone Home Coach
DEVICE = '70:ee:50:7a:5f:10' 

In [15]:
# Fonction pour l'identification
authorization = pyatmo.ClientAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    username=USERNAME,
    password=PASSWORD,
    scope="read_homecoach",
)

In [16]:
# Demande d'accès aux données du capteur
homeData = pyatmo.HomeCoachData(authorization)

In [17]:
# Info sur la station
homeData.stations[DEVICE]

{'_id': '70:ee:50:7a:5f:10',
 'station_name': 'CapteurAmbiance1',
 'date_setup': 1616696398,
 'last_setup': 1616696398,
 'type': 'NHC',
 'last_status_store': 1619863670,
 'module_name': 'Indoor',
 'firmware': 51,
 'wifi_status': 48,
 'reachable': True,
 'co2_calibrating': False,
 'data_type': ['Temperature',
  'CO2',
  'Humidity',
  'Noise',
  'Pressure',
  'health_idx'],
 'place': {'altitude': 53,
  'city': 'Angers',
  'country': 'FR',
  'timezone': 'Europe/Paris',
  'location': [-0.5554129779256776, 47.466872422087654]},
 'dashboard_data': {'time_utc': 1619863670,
  'Temperature': 18.3,
  'CO2': 543,
  'Humidity': 45,
  'Noise': 42,
  'Pressure': 1013.6,
  'AbsolutePressure': 1007.3,
  'health_idx': 0,
  'min_temp': 18.2,
  'max_temp': 18.9,
  'date_max_temp': 1619820088,
  'date_min_temp': 1619848537},
 'modules': [{...}],
 'main_device': '70:ee:50:7a:5f:10'}

In [18]:
# Température et humidité min et max mesurées
homeData.get_min_max_t_h(DEVICE)

(18.2, 19.4, 43, 48)

In [19]:
# Types de données enregistrées
homeData.get_monitored_conditions(DEVICE)

['Temperature',
 'CO2',
 'Humidity',
 'Noise',
 'Pressure',
 'health_idx',
 'wifi_status',
 'reachable']

In [20]:
# Récupération des données sur le temperature, CO2,  à un pas de temps de 5 min
# entrée de la fonction get_data : 'device_id', 'scale', and 'module_type'
pas=1616696550
temperature=homeData.get_data(DEVICE,'5min','temperature',date_begin=str(pas),date_end=str(pas+250000))
co2=homeData.get_data(DEVICE,'5min','co2',date_begin=str(pas),date_end=str(pas+250000))
humidity=homeData.get_data(DEVICE,'5min','humidity',date_begin=str(pas),date_end=str(pas+250000))
noise=homeData.get_data(DEVICE,'5min','noise',date_begin=str(pas),date_end=str(pas+250000))
Pressure=homeData.get_data(DEVICE,'5min','Pressure',date_begin=str(pas),date_end=str(pas+250000))


ts = datetime.datetime.now().timestamp() 

# chargement de toutes les données dans la base
while pas<ts:
    pas=pas+250000
    temperature_temp=homeData.get_data(DEVICE,'5min','temperature',date_begin=str(pas),date_end=str(pas+250000))
    co2_temp=homeData.get_data(DEVICE,'5min','co2',date_begin=str(pas),date_end=str(pas+250000))
    humidity_temp=homeData.get_data(DEVICE,'5min','humidity',date_begin=str(pas),date_end=str(pas+250000))
    noise_temp=homeData.get_data(DEVICE,'5min','noise',date_begin=str(pas),date_end=str(pas+250000))
    Pressure_temp=homeData.get_data(DEVICE,'5min','Pressure',date_begin=str(pas),date_end=str(pas+250000))
    
    temperature['body'].update(temperature_temp['body'])
    co2['body'].update(co2_temp['body'])
    humidity['body'].update(humidity_temp['body'])
    noise['body'].update(noise_temp['body'])
    Pressure['body'].update(Pressure_temp['body'])
    

In [10]:
# NB : première mesure à ne pas prendre en compte (réalisée avant calibrage)
# à faire :
# -> traduire les dates en un format compréhensible
# -> enregistrer le tout en csv par expl

In [21]:
import csv
delimiter=';'
with open('DonnesNetatmo1mai.csv', '+w', encoding='utf-8') as stream:
        writer = csv.writer(stream, delimiter=delimiter, quoting=csv.QUOTE_NONE, quotechar='',  lineterminator='\n')
        writer.writerow(['time' , 'temperature','co2','humidity','noise','Pressure'])
        for t,c,h,n,p in zip(temperature['body'].items(),co2['body'].items(),humidity['body'].items(),noise['body'].items(),Pressure['body'].items()):
            writer.writerow([int(t[0]) , t[1][0],c[1][0],h[1][0],n[1][0],p[1][0]])
